# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [23]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


Then, we define the model, object function and optimizer that we use to classify.

In [31]:
help(torch.Tensor.view)

Help on method_descriptor:

view(...)
    view(*args) -> Tensor
    
    Returns a new tensor with the same data as the :attr:`self` tensor but of a
    different size.
    
    The returned tensor shares the same data and must have the same number
    of elements, but may have a different size. For a tensor to be viewed, the new
    view size must be compatible with its original size and stride, i.e., each new
    view dimension must either be a subspace of an original dimension, or only span
    across original dimensions :math:`d, d+1, \dots, d+k` that satisfy the following
    contiguity-like condition that :math:`\forall i = 0, \dots, k-1`,
    
    .. math::
    
      stride[i] = stride[i+1] \times size[i+1]
    
    Otherwise, :func:`contiguous` needs to be called before the tensor can be
    viewed.
    
    Args:
        args (torch.Size or int...): the desired size
    
    Example::
    
        >>> x = torch.randn(4, 4)
        >>> x.size()
        torch.Size([4, 4])
     

In [45]:
help(nn.Module.parameters)

Help on function parameters in module torch.nn.modules.module:

parameters(self)
    Returns an iterator over module parameters.
    
    This is typically passed to an optimizer.
    
    Yields:
        Parameter: module parameter
    
    Example::
    
        >>> for param in model.parameters():
        >>>     print(type(param.data), param.size())
        <class 'torch.FloatTensor'> (20L,)
        <class 'torch.FloatTensor'> (20L, 1L, 5L, 5L)



In [50]:
class SimpleNet(nn.Module):
# TODO:define model
    def _init_(self):
        super(SimpleNet,self)._init_()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=10,kernel_size=3,stride=1,padding=1)
        self.conv2=nn.Conv2d(inchannels=10,out_channels=20,kernel_size=5,stride=1,padding=2)
        self.dropout=nn.Dropout2d(p=0.2)
        self.fc1=nn.Linear(in_features=980,out_features=50)
        self.fc2=nn.Linear(in_features=50,out_features=10)
        self.max_pool=nn.functional.max_pool2d(kernel_size=2)
        self.relu=nn.function.relu()
    
    def forward(self,x):
        #input is 28*28*1
        #conv1(kernel=3,padding=1,filters=10) 28*28*1->28*28*10
        #max_pool(kernel=2) 28*28*10->14*14*10
        x=self.relu(self.max_pool(self.conv1(x)))
        
        #conv2(kernel=5,padding=2,filters=10) 14*14*10->14*14*20
        #max_pool(kernel=2) 14*14*20->7*7*20
        x = self.relu(self.max_pool2d(self.dropout(self.conv2(x))))
        
        #flatten 7*7*20=980
        x=x.view(-1,980)
        
        #980->50
        x=self.relu(self.fc1(x))
        
        #50->10
        x=self.relu(self.fc2(x))
        
        return nn.functional.log_softmax(x)
    
model = SimpleNet()
param=list(model.parameters())

# TODO:define loss function and optimiter
criterion = nn.functional.nll_loss
optimizer = torch.optim.SGD(param,lr=0.01,momentum=0.5)

ValueError: optimizer got an empty parameter list

Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        
        
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
    
    
    


#### Q5:
Please print the training and testing accuracy.